<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1001]:
import pandas as pd
import numpy as np
import scipy.stats as st
from scipy.stats import chi2
import os
import datetime

In [1004]:
class Get_investment_portfolio():
    """
    Класс генерирующий инвестиционный портфель, на основании переданных бумаг 
    и заданных параметров
    """
    
    
    def __init__(self,data_dir_path,portfolio_income,unrisk_income,start_cash = 100000,
                 confidence_level_normal = 0.01,confidence_level_interval = 0.95,
                 outliers = True, last_income = False, start_portfolio = {}):
        """
        Инициализация параметров класса
        """
        self.outliers = outliers
        self.data_dir_path = data_dir_path
        self.confidence_level_normal = confidence_level_normal
        self.income_data = 0
        self.confidence_level_interval = confidence_level_interval
        self.portfolio_income = portfolio_income
        self.unrisk_income = unrisk_income
        self.last_income = last_income
        self.start_portfolio = start_portfolio
        self.start_cash = start_cash
        

            
            
    def fit_trainsform(self):
        """
        Метод пайплайн, формирующий очередность выполнения методов класса
        """
        self.__basic_load()
        self.__merge()
        if  self.last_income:
            self.__get_income_rebalance()
        self.__calculate_income()
        if self.outliers:
            self.__check_normaltest()
        else:
            self.__check_normaltest_without_outliers()
        if len(self.pass_shares) >= 10:
            self.__get_confidence_interval()
            self.__get_corr_mask()
            self.__format_cov_matrix()
            print("Составленный портфель Тобина:")
            return self.__solve_Tobin()
        else:
            print("Не все акции прошли проверку, попробуй заменить акции, которые не прошли проверку")
    
    
    def __basic_load(self):
        """
        Метод отвечающий за загрузку данных из указанной директории
        """
        # Получаем имена всех фалов, находщихся в указанной директории
        all_files = [name for _,_,name in os.walk(self.data_dir_path) if name != '.DS_Store'][0]
        if '.DS_Store' in all_files:
            index_del = all_files.index('.DS_Store')
            all_files.pop(index_del)
        print(all_files,end = "\n\n")
        # Генерируем словарь имен и значений полученных из фалов
        self.all_data = {name[:-4]:pd.read_csv(self.data_dir_path+"/"+name).loc[:,["Дата","Цена"]] for name in all_files}

        
        
    def __merge(self):
        """
        Метод отвечающий за объединение таблиц в одну
        """
        count = 0
        for dataset in self.all_data.values():
            if count == 0:
                self.total = dataset.copy()
            else:
                self.total = self.total.merge(dataset, on='Дата')
            count+= 1
        # Установим признак "Дата" как индекс всей таблице
        self.total = self.total.set_index("Дата")
        self.total.columns = self.all_data.keys()
           

            
    def __get_income_rebalance(self):
        """
        Метод отвечающий за вычисление доходность за период T1 и вычисление изменений капитала 
        """
        # Вычисление текущих значений цен, выбранных акций и их обработка
        present_price = self.total.iloc[0].apply(lambda x: x.replace(",", "."))
        present_price = present_price.apply(lambda price: float(self.__format_data(price)))
        # Вычисление прошлых значений цен, выбранных акций и их обработка
        past_price = self.total.iloc[1].apply(lambda x: x.replace(",", "."))
        past_price = past_price.apply(lambda price: float(self.__format_data(price)))
        # Вычисление изменения цен в долях
        self.result_income_frac = ((present_price-past_price)/past_price).to_dict()
        # вычисление изменений вложений в каждую акцию 
        self.result_income_value = {delta[0]:delta[1]*self.start_cash*item_before[1] for delta,item_before in zip(self.result_income_frac.items(),self.start_portfolio.items())}
        # Вычисление разницы между стартовым капиталом и текущим 
        delta = round(sum(self.result_income_value.values()),3)
        # Получение текущего капитала
        cur_cash = round(self.start_cash + delta,3)
        # Блок вывода
        print("Динамика акций за период:")
        print("В долях")
        print(self.result_income_frac,end = "\n\n")  
        print("В рублях")
        print(self.result_income_value,end = "\n\n")
        print("Текущий капитал: {}.\n Стартовый капитал: {}.\n Разница: {}".format(cur_cash,self.start_cash,delta),end = "\n\n")
        # Обновление значения капитала
        self.start_cash = cur_cash
    
    
    
    def __format_data(self,replaced_value):
        """
        Метод отвечающий за приведение значений к формату, необходимому для обработки
        Метод заменяет разделитель "," на "." и формирует корректное число
        Принцип работы:  Заменим все запятые на точки, затем начиная со второй точки будем 
        
        """
        # Инициализируем счетчик точек
        count_dot = 0
        # Переберем все значения
        for letter_num in range(1,len(replaced_value)):
                count_dot += replaced_value[-letter_num] == '.'
                if count_dot >= 2:
                        end = len(replaced_value)-letter_num
                        replaced_value = replaced_value[:end] + "" + replaced_value[end + 1:]
                        count_dot-=1              
        return replaced_value
    
    
    
    def __calculate_income(self):
        """
        Метод вычисляющий еженедельную доходность в годовых процентах
        """
        total_week_per_year = 52
        count_weeks = self.total.shape[0]
        # Инициализир пустой датафрейм
        self.income_data = pd.DataFrame()
#         start_price = self.total.iloc[count_weeks-1]
        # Сгенерируем  имена столбцов 
        self.add_income_text = "Income_"
        new_labels_income = [self.add_income_text+name for name in self.total.columns]
        for new_label,old_label in zip(new_labels_income,self.total.columns):
            # Отформатируем стартовое значение
#             main = start_price[old_label].replace(",", ".")
#             main = self.format_data(main)

            for row in range(1,count_weeks):
                # Отформатируем текущее значение
                if "," in str(self.total.iloc[row-1][old_label]):
                    cur_val = self.total.iloc[row-1][old_label].replace(",", ".")
                    cur_val = self.__format_data(cur_val)
                else:
                    cur_val = self.total.iloc[row-1][old_label]
                    
                    
                if "," in str(self.total.iloc[row][old_label]):
                    #Отформатируем следующее значение
                    next_val = self.total.iloc[row][old_label].replace(",", ".")
                    next_val = self.__format_data(next_val)
                else:
                    next_val = self.total.iloc[row][old_label]
                # Вычислим недельную доходность
                self.income_data.loc[row-1,new_label] = ((float(cur_val)- float(next_val))/float(next_val))*100*total_week_per_year


    
    def __check_normaltest(self):
        """
        Метод отвечающий за провекру гипотеза на нормальность выборки
        """
        # Инициализируем словарь для акций, прошедших проверку
        self.pass_shares = {}
        # Инициализируем словарь для акций, не прошедших проверку
        self.fail_shares = {}
        # Переберем все акции из нашей таблицы доходности 
        for label in self.income_data.columns:
            # Получим для каждой выборки p-value
            
            _, p_value = st.normaltest(self.income_data[label])

            # Сравним найденное p-value с уровнем значимости 
            if round(p_value,2) < self.confidence_level_normal:  
                self.fail_shares[label] = p_value
            else:
                self.pass_shares[label] = p_value
        # Проверим все ли акции прошли проверку
        if len(self.pass_shares) >= 10:
            print("Все отлично, минимальное количество акций прошло проверку",end = "\n\n")
            
            top_best_shares = (self.income_data.describe().loc["mean",list(self.pass_shares.keys())]
                                 .sort_values(ascending = False)
                                 .head(len(self.pass_shares))
                                 .index.tolist())
            
            self.income_data = self.income_data.loc[:,top_best_shares]
            
        else:
            # Выведем акции и их p-value, которые не прошли проверку 
            print("Не все акции прошли проверку. Вот неподходящие акции")
            
            print(self.fail_shares,end = "\n\n")
    
    
    
    def __removing_outliers(self,column,frame, count = 0,total_outliers = 0):
        """
        Метод удаления выбросов с помощью межквантильного размаха
        """
        # Получим квантиль 25%
        q25=np.array(frame[column].describe(percentiles=[.25,.75]).loc['25%'])
        # Получим квантиль 75%
        q75=np.array(frame[column].describe(percentiles=[.25,.75]).loc['75%'])
        # Получим первую границу
        first_part=q25-1.5*(q75-q25)
        # Получим 2 границу
        second_part=q75+1.5*(q75-q25)
        # Инициализируем список для индексов, подготовленных к удалению
        index_del= []
        # Переберем все значения и проверим на принадлежность к установленному открезку
        for index, element in enumerate(frame[column]):
            if first_part > element or second_part< element:
                index_del.extend(frame[frame[column] == element].index)
        index_del= list(set(index_del))
        leng = len(index_del)
        count += 1
        
        # Удалим выбранные индексы
        frame = frame.drop(index_del,axis=0)
        if leng > 0:
            frame,leng,total_outliers = self.__removing_outliers(column,frame, count,total_outliers+leng)
        else:
            print('Количество строк, выбранных для удаления {}: {}. Количество итераций {}'.format(column,total_outliers,count),end = "\n\n")
        return frame,leng,total_outliers
    
    
    
    def __check_normaltest_without_outliers(self):
        """
        Метод отвечающий за провекру гипотеза на нормальность выборки с обработкой выбросов 
        """
        # Инициализируем словарь для акций, прошедших проверку
        self.pass_shares = {}
        # Инициализируем словарь для акций, не прошедших проверку
        self.fail_shares = {}
        # Переберем все акции из нашей таблицы доходности 
        for label in self.income_data.columns:
            # Удаление выбросов
            wo_frame,_,_ = self.__removing_outliers(label,self.income_data)
            # Получим для каждой выборки p-value
            _, p_value = st.normaltest(wo_frame[label])

            # Сравним найденное p-value с уровнем значимости 
            if p < self.confidence_level_normal:  
                self.fail_shares[label] = p_value
            else:
                self.pass_shares[label] = p_value
        # Проверим все ли акции прошли проверку
        if len(self.pass_shares) == len(self.income_data.columns):
            print("Все отлично, все акции прошли проверку",end = "\n\n")
            top10_best_shares = self.income_data.describe().loc["mean",list(self.pass_shares.keys())].sort_values(ascending = False).head(10).index.tolist()
            self.income_data = self.income_data.loc[:,top10_best_shares]
            
        else:
            # Выведем акции и их p-value, которые не прошли проверку 
            print("Проверка не пройдена. Вот неподходящие акции")
            print(self.fail_shares,end = "\n\n")
            
    
    
    def __get_confidence_interval(self):
        # Инициализируем индексы, которые в дальнейше преобразуем в мультииндекс
        pre_index = [("estimate_mean","left_side"),
                      ("estimate_mean","right_side"),
                      ("estimate_std","left_side"),
                      ("estimate_std","right_side")]
        
        list_index = np.empty(len(pre_index), dtype=object)
        list_index[:] = pre_index

        index = pd.MultiIndex.from_tuples(list_index)
        # Инициализируем фрейм, в который будем записывать найденные оценки 
        self.period_estimate_table = pd.DataFrame(index = index)
        for label in self.income_data.columns:
            # Вычислим оценку среднего
            mean_estimate =  st.t.interval(self.confidence_level_interval,
                                           len(self.income_data[label])-1,
                                           self.income_data[label].mean(),
                                           self.income_data[label].sem())
            # Добавим найденные данные в таблицу
            self.period_estimate_table.loc[list_index[0],label] = mean_estimate[0]
            self.period_estimate_table.loc[list_index[1],label] = mean_estimate[1]


            # Вычислим оценку отклонения
            # Определим уровни
            p1 = (1 + self.confidence_level_interval)/2
            p2 = (1 - self.confidence_level_interval)/2 
            # Найдем, интересующие нас значения
            value1 = chi2.ppf(p1, self.income_data.shape[0])
            value2 = chi2.ppf(p2, self.income_data.shape[0])
            pre_culc = self.income_data[label].std()*(self.income_data.shape[0])


            # Дополним таблицу, найденными знаениями
            self.period_estimate_table.loc[list_index[2],label] = round((((pre_culc)/value1)**0.5),2)
            self.period_estimate_table.loc[list_index[3],label] = round((((pre_culc)/value2)**0.5),2)

    
    
    def __get_corr_mask(self):
        """
        Метод, который проверят гипотезу на равенство коэффициента коррелции равном 0
        """
        # Инициализируем пустой массив, а котором будет хранить маску
        self.corr_mask = []
        n = self.income_data.describe().loc["count"][0]
        # Переберем все коэффициенты
        for array, i in zip(self.income_data.corr().values,range(0,len(self.income_data.corr()))):
                pre_corr = []
                for ind in range(len(array)):
                    if ind != i:
                        # Найдем наблюдаемое значение
                        t_nabl = (array[ind]*(n-2)**0.5)/(1-array[ind]**2)**0.5
                        # Сравним наблюдаемое значение с критическим 
                        if st.t.ppf(1-0.05/2, n-2) < abs(t_nabl):
                            pre_corr.append(0)
                        else:
                            pre_corr.append(1)
                    else:
                        pre_corr.append(1)
                self.corr_mask.append(pre_corr)
                
    
    
    def __format_cov_matrix(self):
        """
        Метод, обрабатывающий матрицу ковариации, согласно найденной маске корреляции 
        """
        self.total_cov = []
        for array_corr,array_cov in zip(self.corr_mask,self.income_data.cov().values):
            pre_cov = []
            for i in range(len(array_corr)):
                if array_corr[i] == 1:
                    pre_cov.append(array_cov[i])
                else:
                    pre_cov.append(0)
            self.total_cov.append(pre_cov)
    
    
    
    def __solve_Tobin(self):
        """
        Метод, решиающий задачу Тобина
        """
        # Получаем обратную матрицу ковариаций 
        V_reverse = np.linalg.inv(self.total_cov)
        # Получаем вектор доходностей активов
        incomes_vector = self.income_data.describe().loc["mean",:].values
        # Преобразуем вектор доходностей с учетом безрискового актива
        ready_vector = incomes_vector - self.unrisk_income*np.ones(len(incomes_vector))
        # Решаем уравнение Тобина
        D = (ready_vector.T@V_reverse)@ready_vector
        self.portfolio_frac = ((self.portfolio_income-self.unrisk_income)/D).reshape(1,)*V_reverse@ready_vector
        
        portfolio = {label[len(self.add_income_text):]:round(frac,4) for label,frac in zip(self.income_data.columns,self.portfolio_frac)}
        portfolio["unrisk"] = round(1- sum(portfolio.values()),4)
        self.value_portfolio = {key:item*self.start_cash for key,item in portfolio.items()}
        
        print("В долях")
        print(portfolio,end = "\n\n")  
        print("В рублях из ",sum(self.value_portfolio.values()))
        print(self.value_portfolio,end = "\n\n")
        
        return(portfolio)

In [1005]:
obj = Get_investment_portfolio("./data",portfolio_income=40,unrisk_income=3,outliers=True)

In [1006]:
port = obj.fit_trainsform()

['TSM.csv', 'ALRS.csv', 'AMD.csv', 'NVDA.csv', 'YNDX.csv', 'FDX.csv', 'NFLX.csv', 'FIVEDR.csv', 'GOOGL.csv', 'MU.csv']

Все отлично, минимальное количество акций прошло проверку

Составленный портфель Тобина:
В долях
{'AMD': 0.075, 'NVDA': 0.1085, 'TSM': 0.1689, 'MU': 0.0582, 'YNDX': 0.0808, 'GOOGL': 0.1038, 'FIVEDR': 0.0431, 'NFLX': 0.0468, 'FDX': 0.0272, 'ALRS': 0.0005, 'unrisk': 0.2872}

В рублях из  100000.0
{'AMD': 7500.0, 'NVDA': 10850.0, 'TSM': 16890.0, 'MU': 5820.0, 'YNDX': 8080.0, 'GOOGL': 10380.0, 'FIVEDR': 4310.0, 'NFLX': 4680.0, 'FDX': 2720.0, 'ALRS': 50.0, 'unrisk': 28720.0}



In [1007]:
obj_one = Get_investment_portfolio("./data_one",portfolio_income=40,unrisk_income=3,outliers=True,last_income=True,start_portfolio=port)

In [1008]:
port_one = obj_one.fit_trainsform()

['TSM.csv', 'ALRS.csv', 'AMD.csv', 'NVDA.csv', 'YNDX.csv', 'FDX.csv', 'NFLX.csv', 'FIVEDR.csv', 'GOOGL.csv', 'MU.csv']

Динамика акций за период:
В долях
{'TSM': -0.07844285087077418, 'ALRS': 0.026508509541000445, 'AMD': -0.05659745478901533, 'NVDA': -0.081197869560848, 'YNDX': -0.038321678321678355, 'FDX': -0.0005497957901350392, 'NFLX': -0.0025360038502832264, 'FIVEDR': -0.04849660523763336, 'GOOGL': -0.03202780530541306, 'MU': 0.006266490765171585}

В рублях
{'TSM': -588.3213815308063, 'ALRS': 287.6173285198548, 'AMD': -955.9310113864689, 'NVDA': -472.5716008441354, 'YNDX': -309.6391608391611, 'FDX': -5.706880301601706, 'NFLX': -10.930176594720706, 'FIVEDR': -226.96411251212416, 'GOOGL': -87.11563043072353, 'MU': 0.31332453825857925}

Текущий капитал: 97630.751.
 Стартовый капитал: 100000.
 Разница: -2369.249

Все отлично, минимальное количество акций прошло проверку

Составленный портфель Тобина:
В долях
{'AMD': 0.0826, 'NVDA': 0.1142, 'TSM': 0.172, 'MU': 0.0674, 'YNDX': 0.0861, 'G

In [1009]:
obj_two = Get_investment_portfolio("./data_two",portfolio_income=40,
                                   unrisk_income=3,outliers=True,last_income=True,start_cash = 97630.751,start_portfolio=port_one)

In [1010]:
port_two = obj_two.fit_trainsform()

['TSM.csv', 'ALRS.csv', 'AMD.csv', 'NVDA.csv', 'YNDX.csv', 'FDX.csv', 'NFLX.csv', 'FIVEDR.csv', 'GOOGL.csv', 'MU.csv']

Динамика акций за период:
В долях
{'TSM': -0.04081308559631571, 'ALRS': 0.07164389067524125, 'AMD': -0.07087918589516044, 'NVDA': -0.09136315578402432, 'YNDX': -0.018781683703149598, 'FDX': 0.010962671905697526, 'NFLX': -0.0416813584485479, 'FIVEDR': -0.0026503567787971457, 'GOOGL': 0.037172772279676185, 'MU': -0.028405987108051943}

В рублях
{'TSM': -329.12896750487545, 'ALRS': 798.7886704054071, 'AMD': -1190.2379616646567, 'NVDA': -601.198126771097, 'YNDX': -157.87897709703253, 'FDX': 119.0166806925274, 'NFLX': -219.33970748092085, 'FIVEDR': -8.642461179245668, 'GOOGL': 116.86042271621993, 'MU': -21.631723263192345}

Текущий капитал: 96137.359.
 Стартовый капитал: 97630.751.
 Разница: -1493.392

Все отлично, минимальное количество акций прошло проверку

Составленный портфель Тобина:
В долях
{'AMD': 0.085, 'NVDA': 0.1134, 'TSM': 0.1787, 'MU': 0.0714, 'YNDX': 0.0916, 